In [ ]:
# default_exp signIn

# SignIn

> Log into account

In [ ]:
#hide
import src.setEnv 
import requests

In [ ]:
#export 
from src.userTable import UserTable
from awsSchema.apigateway import Response, Event
from copy import deepcopy
from src.saltHashPassword import hash_password, check_password
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from beartype import beartype

## Sign In Input Class

In [ ]:
#export 
@dataclass_json
@dataclass
class SignInInput:
    username: str
    password: str
    
    def checkPassword(self):
        for item in UserTable.username_index.query(self.username):
            return check_password(item.passwordHash, self.password)

## Main Lambda Handler

In [ ]:
#export
class H:
    class ParseInputError(Exception): pass
    class CountUsernameError(Exception): pass
    class InvalidPasswordError(Exception): pass
    class PasswordCheckError(Exception): pass
    class UsernameDoesNotExistError(Exception): pass
    
    @classmethod
    @beartype
    def parseInput(cls, event: dict)->SignInInput:
        try:
            params = Event.parseQuery(event)
            user = SignInInput.from_dict(params)
            return user
        except Exception as e:
            raise cls.ParseInputError(e)
    
    @classmethod
    @beartype
    def checkLoginDetails(cls, user:SignInInput):
        try:
            count = UserTable.username_index.count(user.username)
        except Exception as e:
            raise cls.CountUsernameError(e)
        if count:
            try:
                passwordChecked = user.checkPassword()
            except Exception as e:
                raise cls.PasswordCheckError(e)
            if passwordChecked:
                return True
            else:
                raise cls.InvalidPasswordError
        else: 
            raise cls.UsernameDoesNotExistError

In [ ]:
#export
def signIn(event, *args):
    try:
        user = H.parseInput(event)
        H.checkLoginDetails(user)
        return Response.returnSuccess()
    except H.ParseInputError as e:
        return Response.returnError(f'failed to parse input {e}')
    except H.CountUsernameError as e:
        return Response.returnError(f'failed to count number of user with specified username {e}')
    except H.InvalidPasswordError as e:
        return Response.returnError(f'incorrect password {e}')
    except H.PasswordCheckError as e:
        return Response.returnError(f'failed to check username and password {e}')
    except H.UsernameDoesNotExistError as e:
        return Response.returnError(f'username does not exist or is incorrect {e}')
    except Exception as e:
        return Response.returnError(f' unknown error {e}')

## Tests

### Success 

In [ ]:
input_ = {'username': 'Delgg4', 'password': '12765'}
event = Event.getInput(body = input_)
signIn(event)

{'body': '{"error":"failed to parse input \'username\'"}',
 'statusCode': 400,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

### Failure

#### Invalid Password

In [ ]:
input_1 = {'username': 'Delgg4', 'password': '1276'}
event = Event.getInput(queryStringParameters = input_1)
signIn(event)

{'body': '{"error":"incorrect password "}',
 'statusCode': 400,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

#### Invalid Username

In [ ]:
input_1 = {'username': 'Delgg4', 'password': '1274'}
event = Event.getInput(queryStringParameters = input_1)
signIn(event)

{'body': '{"error":"incorrect password "}',
 'statusCode': 400,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [ ]:
url = 'https://3d4v07deqh.execute-api.ap-southeast-1.amazonaws.com/Prod/signin'
url2 = 'https://sje65cwtekbnrn7w3l66djb2fi0lvmkt.lambda-url.ap-southeast-1.on.aws'
params = {'username': 'Delgg4', 'password': '12765'}
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}
paramsGetInput = Event.getInput(body = params)
r = requests.get(url2, params=params)
print(r.text)
print(r.status_code)

{"error":"failed to parse input 'body'"}
400


In [ ]:
item = UserTable.username_index.query('Delg45678')
for i in item:
    
    passwordHash = i.userId
    print(passwordHash)
    

15fdac0b-6f83-4726-a2b6-ac825ff6e57e
